In [86]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from google.colab import drive

In [87]:
drive.mount('/content/drive')
df = pd.read_csv('drive/My Drive/Dataset/fake_news_train.csv')
df.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [88]:
# Drop the nan values
df = df.dropna()

In [89]:
X = df.drop('label', axis = 1)

In [90]:
y = df['label']

In [91]:
X.shape

(18285, 4)

In [92]:
y.shape

(18285,)

In [93]:
import tensorflow as tf

In [94]:
tf.__version__

'2.8.2'

In [95]:
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [96]:
# Vocubolary size
voc_size = 5000

**Onehot Representation**

In [97]:
messages = X.copy()

In [98]:
messages.reset_index(inplace = True)

In [99]:
messages['title']

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
18280    Rapper T.I.: Trump a ’Poster Child For White S...
18281    N.F.L. Playoffs: Schedule, Matchups and Odds -...
18282    Macy’s Is Said to Receive Takeover Approach by...
18283    NATO, Russia To Hold Parallel Exercises In Bal...
18284                            What Keeps the F-35 Alive
Name: title, Length: 18285, dtype: object

In [100]:
messages['title'][3]

'15 Civilians Killed In Single US Airstrike Have Been Identified'

In [101]:
import nltk
import re
from nltk.corpus import stopwords

In [102]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [103]:
# Datset preprocesisng
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  print(i)
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [104]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [105]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]
onehot_rep

[[4013, 2489, 1242, 3074, 3255, 3863, 956, 476, 4250, 1409],
 [379, 104, 1587, 1160, 2891, 2927, 301],
 [4486, 3846, 3091, 282],
 [459, 4891, 531, 2215, 908, 1889],
 [3727, 2891, 762, 3323, 4837, 3337, 2891, 2726, 4550, 1389],
 [1554,
  4501,
  1392,
  2244,
  4182,
  4413,
  69,
  4690,
  3733,
  4458,
  2650,
  4457,
  3015,
  4858,
  301],
 [1523, 3526, 1179, 681, 3039, 521, 3390, 3513, 626, 3072, 2338],
 [4727, 101, 4462, 3415, 2073, 4335, 4413, 300, 626, 3072, 2338],
 [1088, 949, 1023, 1163, 4664, 1178, 4894, 4010, 4413, 4644],
 [1512, 2959, 761, 3296, 3913, 1626, 2542, 4615],
 [677, 3521, 2726, 1695, 2583, 2683, 3919, 3777, 2563, 556, 1524],
 [2215, 469, 3255, 1178, 4413, 2073],
 [1491, 3480, 1994, 165, 1752, 1646, 3136, 3728, 4022],
 [299, 3925, 2684, 1432, 2123, 2156, 453, 626, 3072, 2338],
 [2103, 2130, 4505, 3871, 2719, 626, 3072, 2338],
 [471, 3744, 2062, 3231, 1357, 2946, 1838, 3756, 3477, 1312],
 [1979, 2449, 104],
 [3874, 3529, 2687, 966, 4413, 3543, 503, 301],
 [2561, 44

**Embedding Representation**

In [106]:
sent_length = 20
embedded_docs = pad_sequences(onehot_rep, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  476 4250 1409]
 [   0    0    0 ... 2891 2927  301]
 [   0    0    0 ... 3846 3091  282]
 ...
 [   0    0    0 ...  626 3072 2338]
 [   0    0    0 ...  555 2238 4559]
 [   0    0    0 ... 4684 3053 3458]]


In [107]:
len(embedded_docs)

18285

In [108]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4013,
       2489, 1242, 3074, 3255, 3863,  956,  476, 4250, 1409], dtype=int32)

In [109]:
# Creating Model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_4 (LSTM)               (None, 100)               56400     
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [110]:
len(embedded_docs), y.shape

(18285, (18285,))

In [111]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [112]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [113]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final,y_final,test_size=0.33,random_state = 42)

**Model Training**

In [114]:
model.fit(X_train,y_train,validation_data=(X_test,y_test), epochs = 10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 4s 9ms/step - loss: 0.3425 - accuracy: 0.8393 - val_loss: 0.1925 - val_accuracy: 0.9171
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1399 - accuracy: 0.9458 - val_loss: 0.1957 - val_accuracy: 0.9211
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0988 - accuracy: 0.9610 - val_loss: 0.2051 - val_accuracy: 0.9170
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0701 - accuracy: 0.9749 - val_loss: 0.2333 - val_accuracy: 0.9160
Epoch 5/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0495 - accuracy: 0.9831 - val_loss: 0.2635 - val_accuracy: 0.9105
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0271 - accuracy: 0.9916 - val_loss: 0.3901 - val_accuracy: 0.9142
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0152 - accuracy: 0.9951 - val_loss: 0.4295 - val_accuracy: 0.9138
Epoch 

**Adding Dropout**

In [123]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

**Performance Metrics and Accuracy**

In [125]:
model.fit(X_train,y_train,validation_data=(X_test,y_test), epochs = 10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 3s 10ms/step - loss: 0.3438 - accuracy: 0.8306 - val_loss: 0.1901 - val_accuracy: 0.9195
Epoch 2/10
192/192 [==============================] - 1s 8ms/step - loss: 0.1497 - accuracy: 0.9385 - val_loss: 0.1906 - val_accuracy: 0.9218
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1060 - accuracy: 0.9602 - val_loss: 0.2069 - val_accuracy: 0.9211
Epoch 4/10
192/192 [==============================] - 1s 8ms/step - loss: 0.0747 - accuracy: 0.9719 - val_loss: 0.2482 - val_accuracy: 0.9208
Epoch 5/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0527 - accuracy: 0.9817 - val_loss: 0.2483 - val_accuracy: 0.9195
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0345 - accuracy: 0.9891 - val_loss: 0.2942 - val_accuracy: 0.9225
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0262 - accuracy: 0.9910 - val_loss: 0.3473 - val_accuracy: 0.9150
Epoch

In [126]:
score, acc = model.evaluate(X_test, y_test,verbose=2, batch_size= 64)
print('test accuracy:', acc)

95/95 - 0s - loss: 0.4349 - accuracy: 0.9110 - 241ms/epoch - 3ms/step
test accuracy: 0.9110190272331238
